## Preparation

Import the modules and removed previous database (if any)

In [1]:
import sqlite3
import pandas as pd
import os

In [2]:
# IMPORTANT NOTE
# Note that your path to the database may be different
try:
    os.remove('../dataverse_files/airline2_python.db')
except OSError:
    pass


## Create database 

In [3]:
# Note that your path to the database may be different 
conn = sqlite3.connect('../dataverse_files/airline2_python.db')

OperationalError: unable to open database file

## Create tables 

First create the tables for the airports, carrier and plane-data dataset 

In [14]:
# Note that your path to the data may be different 
airports = pd.read_csv("../dataverse_files/airports.csv")
carriers = pd.read_csv("../dataverse_files/carriers.csv")
planes = pd.read_csv("../dataverse_files/plane-data.csv")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

Then create the table for ontime, which requires several csv files

In [15]:
c = conn.cursor()

In [16]:
c.execute('''
CREATE TABLE ontime (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')

conn.commit()

In [17]:
for year in range(2000, 2006):
    print('Processing: ', year)
    ontime = pd.read_csv("../dataverse_files/"+str(year)+".csv")    
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)

conn.commit()

Processing:  2000
Processing:  2001
Processing:  2002
Processing:  2003
Processing:  2004
Processing:  2005


## Run queries 

Query 1: Find model that has the lowest associated average departure delay

In [27]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

print(c.fetchone()[0], "has the lowest associated average departure delay.")

#x = c.fetchall()  #alternatively you can use fetchall()
#print(x[0][0],"has the highest number of inbound flights (excluding canceled flights) at", x[0][1])
#print(x[:5])

737-2Y5 has the lowest associated average departure delay.


Query 2: Find the city that has the highest number of inbound flights (excluding canceled flights)

In [25]:
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

print(c.fetchone()[0], "has the highest number of inbound flights (excluding canceled flights)")

Chicago has the highest number of inbound flights (excluding canceled flights)


Query 3: Find the carrier that has the highest number of canceled flights

In [28]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
GROUP BY carriers.Description
ORDER BY total DESC
''')

print(c.fetchone()[0],"has the highest number of canceled flights")

Delta Air Lines Inc. has the highest number of canceled flights


Query 4: Find the carrier that has the highest number of canceled flights, relative to their number of total flights

In [29]:
c.execute('''
SELECT
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')

print(c.fetchone()[0], "has the highest number of canceled flights, relative to their number of total flights")

American Eagle Airlines Inc. has the highest number of canceled flights, relative to their number of total flights


In [30]:
# Shortened version for Q4
c.execute('''
SELECT carriers.Description, AVG(Cancelled) AS ratio
FROM carriers INNER JOIN ontime ON ontime.UniqueCarrier = carriers.Code
GROUP BY carriers.Description
ORDER BY ratio DESC
''')

print(c.fetchone()[0], "has the highest number of canceled flights, relative to their number of total flights")

American Eagle Airlines Inc. has the highest number of canceled flights, relative to their number of total flights


Remember to close the connection

In [10]:
conn.close()